## 加了L1

In [1]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.mp(F.relu(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)



model = Net()
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        regularization_loss = 0
        for param in model.parameters():
            regularization_loss += torch.mean(abs(param))
        
        loss = criterion(output, target)
        loss += 0.01 * regularization_loss # lamda = 0.01
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        #test_loss += F.nll_loss(output, target, reduction='sum').item()
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\raw

1:	Test set: Average loss: 0.0033, Accuracy: 9398/10000 (94%)
2:	Test set: Average loss: 0.0020, Accuracy: 9633/10000 (96%)
3:	Test set: Average loss: 0.0016, Accuracy: 9687/10000 (97%)
4:	Test set: Average loss: 0.0014, Accuracy: 9735/10000 (97%)
5:	Test set: Average loss: 0.0013, Accuracy: 9728/10000 (97%)
6:	Test set: Average loss: 0.0012, Accuracy: 9762/10000 (98%)
7:	Test set: Average loss: 0.0011, Accuracy: 9777/10000 (98%)
8:	Test set: Average loss: 0.0010, Accuracy: 9800/10000 (98%)
9:	Test set: Average loss: 0.0010, Accuracy: 9804/10000 (98%)
10:	Test set: Average loss: 0.0010, Accuracy: 9792/10000 (98%)
11:	Test set: Average loss: 0.0010, Accuracy: 9800/10000 (98%)
12:	Test set: Average loss: 0.0009, Accuracy: 9805/10000 (98%)
13:	Test set: Average loss: 0.0009, Accuracy: 9813/10000 (98%)
14:	Test set: Average loss: 0.0008, Accuracy: 9818/10000 (98%)
15:	Test set: Average loss: 0.0009, Accuracy: 9831/1

## 加了L2

In [2]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.mp(F.relu(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)


model = Net()
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')

1:	Test set: Average loss: 0.0032, Accuracy: 9370/10000 (94%)
2:	Test set: Average loss: 0.0020, Accuracy: 9619/10000 (96%)
3:	Test set: Average loss: 0.0016, Accuracy: 9693/10000 (97%)
4:	Test set: Average loss: 0.0014, Accuracy: 9718/10000 (97%)
5:	Test set: Average loss: 0.0012, Accuracy: 9761/10000 (98%)
6:	Test set: Average loss: 0.0012, Accuracy: 9765/10000 (98%)
7:	Test set: Average loss: 0.0011, Accuracy: 9791/10000 (98%)
8:	Test set: Average loss: 0.0010, Accuracy: 9787/10000 (98%)
9:	Test set: Average loss: 0.0010, Accuracy: 9807/10000 (98%)
10:	Test set: Average loss: 0.0009, Accuracy: 9816/10000 (98%)
11:	Test set: Average loss: 0.0009, Accuracy: 9817/10000 (98%)
12:	Test set: Average loss: 0.0009, Accuracy: 9819/10000 (98%)
13:	Test set: Average loss: 0.0009, Accuracy: 9834/10000 (98%)
14:	Test set: Average loss: 0.0009, Accuracy: 9821/10000 (98%)
15:	Test set: Average loss: 0.0008, Accuracy: 9834/10000 (98%)
16:	Test set: Average loss: 0.0008, Accuracy: 9830/10000 (98%)
1

# Normalization Input Data

In [3]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
])

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=trans,
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=trans)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.mp(F.relu(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)

model = Net()
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')

1:	Test set: Average loss: 0.0019, Accuracy: 9609/10000 (96%)
2:	Test set: Average loss: 0.0013, Accuracy: 9742/10000 (97%)
3:	Test set: Average loss: 0.0011, Accuracy: 9795/10000 (98%)
4:	Test set: Average loss: 0.0009, Accuracy: 9802/10000 (98%)
5:	Test set: Average loss: 0.0009, Accuracy: 9836/10000 (98%)
6:	Test set: Average loss: 0.0008, Accuracy: 9838/10000 (98%)
7:	Test set: Average loss: 0.0008, Accuracy: 9833/10000 (98%)
8:	Test set: Average loss: 0.0007, Accuracy: 9865/10000 (99%)
9:	Test set: Average loss: 0.0007, Accuracy: 9861/10000 (99%)
10:	Test set: Average loss: 0.0007, Accuracy: 9861/10000 (99%)
11:	Test set: Average loss: 0.0007, Accuracy: 9859/10000 (99%)
12:	Test set: Average loss: 0.0006, Accuracy: 9868/10000 (99%)
13:	Test set: Average loss: 0.0006, Accuracy: 9869/10000 (99%)
14:	Test set: Average loss: 0.0006, Accuracy: 9865/10000 (99%)
15:	Test set: Average loss: 0.0006, Accuracy: 9879/10000 (99%)
16:	Test set: Average loss: 0.0006, Accuracy: 9877/10000 (99%)
1

# With Batch Norm

In [4]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
])

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=trans,
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=trans)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv1_bn = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_bn = nn.BatchNorm2d(20)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc1_bn = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 10)
        self.fc2_bn = nn.BatchNorm1d(10)

    def forward(self, x, y=None):
        x = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)), 2))
        x = F.max_pool2d(self.conv2_drop(self.conv2_bn(self.conv2(x))), 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = F.dropout(x, training=self.training)
        x = self.fc2_bn(self.fc2(x))
        return F.log_softmax(x, dim=0)
model = Net()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 31):
    train(epoch)
    test(epoch)
print('done')

1:	Test set: Average loss: 0.0053, Accuracy: 9615/10000 (96%)
2:	Test set: Average loss: 0.0032, Accuracy: 9711/10000 (97%)
3:	Test set: Average loss: 0.0025, Accuracy: 9754/10000 (98%)
4:	Test set: Average loss: 0.0019, Accuracy: 9792/10000 (98%)
5:	Test set: Average loss: 0.0017, Accuracy: 9814/10000 (98%)
6:	Test set: Average loss: 0.0015, Accuracy: 9818/10000 (98%)
7:	Test set: Average loss: 0.0014, Accuracy: 9843/10000 (98%)
8:	Test set: Average loss: 0.0013, Accuracy: 9840/10000 (98%)
9:	Test set: Average loss: 0.0012, Accuracy: 9845/10000 (98%)
10:	Test set: Average loss: 0.0011, Accuracy: 9865/10000 (99%)
11:	Test set: Average loss: 0.0011, Accuracy: 9848/10000 (98%)
12:	Test set: Average loss: 0.0010, Accuracy: 9865/10000 (99%)
13:	Test set: Average loss: 0.0010, Accuracy: 9861/10000 (99%)
14:	Test set: Average loss: 0.0010, Accuracy: 9866/10000 (99%)
15:	Test set: Average loss: 0.0010, Accuracy: 9858/10000 (99%)
16:	Test set: Average loss: 0.0009, Accuracy: 9865/10000 (99%)
1

# ADAM

In [5]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
])

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=trans,
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=trans)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv1_bn = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_bn = nn.BatchNorm2d(20)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc1_bn = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 10)
        self.fc2_bn = nn.BatchNorm1d(10)

    def forward(self, x, y=None):
        x = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)), 2))
        x = F.max_pool2d(self.conv2_drop(self.conv2_bn(self.conv2(x))), 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = F.dropout(x, training=self.training)
        x = self.fc2_bn(self.fc2(x))
        return F.log_softmax(x, dim=0)

model = Net()
if torch.cuda.device_count() > 2:
    print("On", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model = model.to(device)
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 31):
    train(epoch)
    test(epoch)
print('done')

1:	Test set: Average loss: 0.0020, Accuracy: 9749/10000 (97%)
2:	Test set: Average loss: 0.0016, Accuracy: 9768/10000 (98%)
3:	Test set: Average loss: 0.0015, Accuracy: 9790/10000 (98%)
4:	Test set: Average loss: 0.0016, Accuracy: 9761/10000 (98%)
5:	Test set: Average loss: 0.0014, Accuracy: 9792/10000 (98%)
6:	Test set: Average loss: 0.0016, Accuracy: 9760/10000 (98%)
7:	Test set: Average loss: 0.0015, Accuracy: 9770/10000 (98%)
8:	Test set: Average loss: 0.0015, Accuracy: 9764/10000 (98%)
9:	Test set: Average loss: 0.0015, Accuracy: 9784/10000 (98%)
10:	Test set: Average loss: 0.0015, Accuracy: 9784/10000 (98%)
11:	Test set: Average loss: 0.0015, Accuracy: 9775/10000 (98%)
12:	Test set: Average loss: 0.0015, Accuracy: 9777/10000 (98%)
13:	Test set: Average loss: 0.0013, Accuracy: 9791/10000 (98%)
14:	Test set: Average loss: 0.0014, Accuracy: 9785/10000 (98%)
15:	Test set: Average loss: 0.0013, Accuracy: 9776/10000 (98%)
16:	Test set: Average loss: 0.0013, Accuracy: 9808/10000 (98%)
1

# 換上預選的model

In [6]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from tqdm import tqdm

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 512

trans = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=trans,
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=trans)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

num_classes = 10

model = models.densenet201(pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 10)

if torch.cuda.device_count() > 1:
    print("On", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in tqdm(test_loader):
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')

c:\Users\joshhu\anaconda3\envs\chscht\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\joshhu\anaconda3\envs\chscht\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to C:\Users\joshhu/.cache\torch\hub\checkpoints\densenet201-c1103571.pth


  0%|          | 0.00/77.4M [00:00<?, ?B/s]

100%|██████████| 20/20 [00:02<00:00,  8.10it/s]


1:	Test set: Average loss: 0.0001, Accuracy: 9803/10000 (98%)


100%|██████████| 20/20 [00:02<00:00,  8.08it/s]


...

18:	Test set: Average loss: 0.0001, Accuracy: 9931/10000 (99%)


100%|██████████| 20/20 [00:02<00:00,  7.78it/s]


19:	Test set: Average loss: 0.0001, Accuracy: 9930/10000 (99%)


100%|██████████| 20/20 [00:02<00:00,  7.80it/s]

20:	Test set: Average loss: 0.0001, Accuracy: 9927/10000 (99%)
done
CPU times: total: 2min 14s
Wall time: 8min 7s
